## Preprocessing

First attempt of optimizing the model accancy by dropping few columns and adding the number of epochs.

In [7]:
# Import our dependencies
# import os 
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np


#  Import and read the charity_data.csv.
import pandas as pd 
df = pd.read_csv("charity_data.csv")
df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [27]:
# Second Attempt - optimising the model by removing two more columns from data on 50 more epochs and adding one more hidden layer
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5779 - accuracy: 0.7229 - 473ms/epoch - 2ms/step
Loss: 0.5778949856758118, Accuracy: 0.7229154706001282


In [33]:
# First attempt - optimising the model by removing two more columns from data on 50 more epochs 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5758 - accuracy: 0.7272 - 393ms/epoch - 1ms/step
Loss: 0.5758264064788818, Accuracy: 0.7272303104400635


In [39]:
# third attempt - optimising the model by removing two more columns from data on 50 more epochs and removing 10 nodes frpm 1st hidden layer and adding 40 nodes in second
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5781 - accuracy: 0.7198 - 389ms/epoch - 1ms/step
Loss: 0.578086256980896, Accuracy: 0.7197667360305786


In [8]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME', 'ORGANIZATION','SPECIAL_CONSIDERATIONS'
application_df = df[[
    "APPLICATION_TYPE",
    "AFFILIATION",
    "CLASSIFICATION",
    "USE_CASE",
    "STATUS",
    "INCOME_AMT",
    "ASK_AMT",
    "IS_SUCCESSFUL"
]]
application_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  STATUS  \
0              T10       Independent          C1000    ProductDev       1   
1               T3       Independent          C2000  Preservation       1   
2               T5  CompanySponsored          C3000    ProductDev       1   
3               T3  CompanySponsored          C2000  Preservation       1   
4               T3       Independent          C1000     Heathcare       1   

      INCOME_AMT  ASK_AMT  IS_SUCCESSFUL  
0              0     5000              1  
1         1-9999   108590              1  
2              0     5000              0  
3    10000-24999     6692              1  
4  100000-499999   142590              1

In [9]:
# Determine the number of unique values in each column.
n = application_df.nunique(axis=0)
  
print("\n",
      n)


 APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [10]:
# Look at APPLICATION_TYPE value counts for binning
print(application_df['APPLICATION_TYPE'].value_counts())


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [11]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_count = application_df['APPLICATION_TYPE'].copy()
app_count = app_count.value_counts()
application_index = app_count[(app_count < 528)].copy()
application_types_to_replace = application_index.index
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Index(['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [12]:
# Look at CLASSIFICATION value counts for binning
class_count= application_df['CLASSIFICATION'].value_counts()
class_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [13]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count[class_count>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [14]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

class_index =class_count[(class_count < 1883)].copy()
classifications_to_replace = class_index.index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies

STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       0   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1                     0                    0   
1                         0                     0                    1   
2                         0                     0                    0   
3                         0                     0                    1   
4                         0                     0                    1   
...                     ...                   ...                  ...   
34294                     0                     0                    0   
34295                     0                     0                    0   
34296                     0                     0                    1   
34297                     0                     0                    0   
34298                     0                     0                    1   

       APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                        0                    0                    0  ...   
1                        0                    0                    0  ...   
2                        0                    1                    0  ...   
3                        0                    0                    0  ...   
4                        0                    0                    0  ...   
...                    ...                  ...                  ...  ...   
34294                    1                    0                    0  ...   
34295                    1                    0                    0  ...   
34296                    0                    0                    0  ...   
34297                    0                    1                    0  ...   
34298                    0                    0                    0  ...   

       USE_CASE_ProductDev  INCOME_AMT_0  INCOME_AMT_1-9999  \
0                        1             1                  0   
1                        0             0                  1   
2                        1             1                  0   
3                        0             0                  0   
4                        0             0                  0   
...                    ...           ...                ...   
34294                    1             1                  0   
34295                    1             1                  0   
34296                    0             1                  0   
34297                    1             1                  0   
34298                    0             0                  0   

       INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  INCOME_AMT_10M-50M  \
0                           0                         0                   0   
1                           0                         0                   0   
2                           0                         0                   0   
3                           1                         0                   0   
4                           0                         1                   0   
...                       ...                       ...                 ...   
34294                       0                         0                   0   
34295      

In [16]:
# Split our preprocessed data into our features and target arrays
X= application_df_dummies.drop(columns=["IS_SUCCESSFUL"])
y = application_df_dummies['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimensions = len(X_train_scaled[0])
hidden_layer_1 = 70
hidden_layer_2 = 70

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim=input_dimensions))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))



# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 70)                2660      
                                                                 
 dense_11 (Dense)            (None, 70)                4970      
                                                                 
 dense_12 (Dense)            (None, 1)                 71        
                                                                 
Total params: 7,701
Trainable params: 7,701
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])



In [36]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5762 - accuracy: 0.7190
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5647 - accuracy: 0.7247
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7250
Epoch 4/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5611 - accuracy: 0.7259
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - accuracy: 0.7257
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5592 - accuracy: 0.7276
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5587 - accuracy: 0.7269
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5578 - accuracy: 0.7260
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7278
Epoch 10/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accura

In [38]:
# Evaluate the model using the test data on 150 epochs
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5781 - accuracy: 0.7198 - 430ms/epoch - 2ms/step
Loss: 0.578086256980896, Accuracy: 0.7197667360305786


In [40]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")